### Оценка переводов facebook/nllb-200-distilled-600M и google translate

In [2]:
import pandas as pd
df_en = pd.read_csv('wiki_en_jpeg.csv', usecols=["image_url", "caption_reference_description"])

In [3]:
df_ru = pd.read_csv('wiki_ru_jpeg.csv', usecols=["image_url", "caption_reference_description"])

Merge по изображениям

In [6]:
df_en_ru = df_en.merge(df_ru, on=['image_url'], how='left', indicator=True)
df_en_ru

,image_url,caption_reference_description_x,caption_reference_description_y,_merge
0,https://upload.wikimedia.org/wikipedia/commons...,Mounted skeleton of Oxydactylus longipes in th...,NaN,left_only
1,https://upload.wikimedia.org/wikipedia/commons...,"Samuel W. Gould, US Representative from Maine",NaN,left_only
2,https://upload.wikimedia.org/wikipedia/commons...,Hungarian MEP,NaN,left_only
3,https://upload.wikimedia.org/wikipedia/commons...,A captive decoy,Подсадная птица,both
4,https://upload.wikimedia.org/wikipedia/commons...,Blue plaque dedicated to Mark Gertler,NaN,left_only
...,...,...,...,...
2392091,https://upload.wikimedia.org/wikipedia/commons...,City magistrate,NaN,left_only
2392092,https://upload.wikimedia.org/wikipedia/commons...,Carrington gives CN 20th-anniversary speech,NaN,left_only
2392093,https://upload.wikimedia.org/wikipedia/commons...,"Long Quay, Kinsale, c. 1900",NaN,left_only
2392094,https://upload.wikimedia.org/wikipedia/commons...,"Crystal Peaks bus station, March 2015",NaN,left_only


In [7]:
mask = (df_en_ru['_merge'] == 'left_only') #изображения, которые есть только в df_en
mask.value_counts()

_merge
True     2109916
False     282180
Name: count, dtype: int64

In [8]:
import torch
torch.cuda.is_available()

False

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
from tqdm import tqdm

Перевод с помощью "facebook/nllb-200-distilled-600M"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [12]:
model.to(device)
None

Проверим качество перевода на 100 предложений

In [ ]:
tokenizer.src_lang = "en_XX"
df_mask = df_en_ru[mask]
to_translate = list(df_mask['caption_reference_description_x'].iloc[0:100])
translated_nllb = []
model.eval()
with torch.no_grad():
    for row in tqdm(to_translate):
        encoded_en = tokenizer(row, return_tensors="pt").to(device)
        generated_tokens = model.generate(
            **encoded_en,
            forced_bos_token_id=tokenizer.lang_code_to_id["rus_Cyrl"]
        )
        translated_nllb.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

Перевод с помощью mtranslate

In [14]:
from mtranslate import translate

In [ ]:
df_mask = df_en_ru[mask]
to_translate = list(df_mask['caption_reference_description_x'].iloc[0:100])
translated_m = []
for row in tqdm(to_translate):
    translated_m.append(translate(row, 'ru'))

Оценка с помощью COMET-QE (Z-score)

In [16]:
from comet import download_model, load_from_checkpoint

model_path = download_model("wmt21-comet-qe-mqm")
model = load_from_checkpoint(model_path)
data = [{"src": s, "mt": m} for s, m in zip(to_translate, translated_nllb)]
model_output = model.predict(data, batch_size=8, gpus=1)
score_nllb = model_output.system_score
score_nllb

wmt21-comet-qe-mqm is already in cache.
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\dasha\.cache\torch\unbabel_comet\wmt21-comet-qe-mqm\checkpoints\model.ckpt`
Encoder model frozen.
c:\Users\dasha\miniconda3\envs\classical-ml\Lib\site-packages\pytorch_lightning\core\saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\dasha\miniconda3\envs\classical-ml\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
Predicting DataLoader 0: 100%|██████████| 13/13 [00:12<00:0

0.1015434791147709

In [17]:
data = [{"src": s, "mt": m} for s, m in zip(to_translate, translated_m)]
model_output = model.predict(data, batch_size=8, gpus=1)
score_m = model_output.system_score
score_m

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


c:\Users\dasha\miniconda3\envs\classical-ml\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
Predicting DataLoader 0: 100%|██████████| 13/13 [00:13<00:00,  1.05s/it]


0.10699754532426596

Оценка с помощью COMET-Kiwi (между 0 и 1)

In [18]:
from huggingface_hub import notebook_login

In [19]:
notebook_login()

In [ ]:
from comet import download_model, load_from_checkpoint
model_path = download_model("Unbabel/wmt22-cometkiwi-da")
model = load_from_checkpoint(model_path)

In [21]:
data = [{"src": s, "mt": m} for s, m in zip(to_translate, translated_nllb)]
model_output = model.predict(data, batch_size=8, gpus=1)
score_nllb = model_output.system_score
score_nllb

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\dasha\miniconda3\envs\classical-ml\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
Predicting DataLoader 0: 100%|██████████| 13/13 [00:10<00:00,  1.21it/s]


0.7992514678835869

In [22]:
data = [{"src": s, "mt": m} for s, m in zip(to_translate, translated_m)]
model_output = model.predict(data, batch_size=8, gpus=1)
score_m = model_output.system_score
score_m

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\dasha\miniconda3\envs\classical-ml\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
Predicting DataLoader 0: 100%|██████████| 13/13 [00:10<00:00,  1.18it/s]


0.8322649028897285